# Import

In [1]:
import pandas as pd
import numpy as np
import os
import random

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from catboost import Pool, CatBoostRegressor, cv

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from optuna.integration.mlflow import MLflowCallback

import warnings
warnings.filterwarnings('ignore')

# Seed hold

In [2]:
def seed_all(seed):

    print("Using Seed Number {}".format(seed))

    os.environ["PYTHONHASHSEED"] = str(seed)  
    np.random.seed(seed) 
    random.seed(seed)
    
seed = 1215
seed_all(seed=seed)

Using Seed Number 1215


# Data load

In [3]:
def load_data(df):

    # make month & day
    df["month"] = df["base_date"].apply(lambda x : str(x)[4:6])
    df["day"] = df["base_date"].apply(lambda x : str(x)[6:8])

    # set category
    globals()['cat_features'] = ['base_date', 'day_of_week', 'base_hour', 'lane_count', 'road_rating', 
                'multi_linked', 'connect_code', 'maximum_speed_limit', 'weight_restricted', 
                'height_restricted', 'road_type', 'start_turn_restricted',
                'end_turn_restricted', 'holidays', 'month', 'day', 'road_name', 'start_node_name', 'end_node_name']

    # set numeric          
    globals()['numerial_features'] = ['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude', '제주 강수량', 
                        '제주 적설', '고산 강수량', '고산 적설', '성산 강수량', '성산 적설', 
                        '서귀포 강수량', '서귀포 적설',]
    # to datetime
    df['시간'] = pd.to_datetime(df['시간'])

    # cat feature to string
    for col in cat_features:
        df[col] = df[col].astype(str)
    
    return df

train = load_data(pd.read_csv('train_for_cbr.csv'))
test = load_data(pd.read_csv('test_for_cbr.csv'))

# seperate X, y
X_train = train.drop(['target'], axis=1)
y_train = train['target']

# CBR Optuna

## hyperparameter search

hyper parameter 찾고 feature importance로 feature 버려보기, engineering

### using cross validation(데이터의 양이 많아 너무 많은 시간 소요)

#### make pool

In [4]:
# make Pool
train_pool = Pool(X_train, y_train, cat_features=cat_features, timestamp=X_train['시간'])
test_pool = Pool(test, cat_features=cat_features, timestamp=test['시간'])

#### search

In [14]:
class Objective(object):
    def __init__(self, pool):
        self.pool = pool

    def __call__(self, trial):
        pool = self.pool
        
        CBR_params = {# prefix
                 'task_type' : 'GPU',
                 'random_seed' : seed,
                 'iterations' : 3000,
                 'early_stopping_rounds' : 50,

                 # found
                 # 'loss_function' : trial.suggest_categorical("loss_function", ['MAPE', 'Poisson', 'RMSE']),
                 'loss_function' : 'RMSE',

                 # trial
                 'learning_rate' : trial.suggest_uniform("learning_rate", 1e-4, 3e-1),
                 'depth' : trial.suggest_int("depth", 2, 16),
                 'l2_leaf_reg' : trial.suggest_float("l2_leaf_reg", 1e-5, 5e-1, log=True),
                 }
        
        cv_result = cv(pool=pool,
                       params=CBR_params,
                       nfold=5,
                       verbose=False,)
        
        rmsle = cv_result["test-RMSE-mean"].min()
        
        return rmsle

CBR_objective = Objective(train_pool)

# random sampler
sampler = TPESampler(seed=seed)

CBR_study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.HyperbandPruner(max_resource="auto"),
    sampler=sampler
    )

[I 2022-10-14 04:14:47,086] A new study created in memory with name: CBR0


In [15]:
def make_mlflow_callback():
    cb = MLflowCallback(
        tracking_uri="mlruns",
        metric_name="RMSLE"
    )
    return cb

mlflow_cb = make_mlflow_callback()
CBR_study.optimize(CBR_objective, n_trials=30, callbacks=[mlflow_cb])

# data 폴더로 가서 mlflow ui 실행
# http://localhost:5000/ 로 접속

Training on fold [0/5]
bestTest = 4.888740913
bestIteration = 2999
Training on fold [1/5]
bestTest = 4.916484654
bestIteration = 2998
Training on fold [2/5]
bestTest = 4.980818262
bestIteration = 2999
Training on fold [3/5]
bestTest = 4.970064378
bestIteration = 2999
Training on fold [4/5]


[I 2022-10-14 06:40:59,890] Trial 0 finished with value: 4.942627197981751 and parameters: {'learning_rate': 0.1344064063808722, 'depth': 4, 'l2_leaf_reg': 0.0008103931299344912}. Best is trial 0 with value: 4.942627197981751.
2022/10/14 06:40:59 INFO mlflow.tracking.fluent: Experiment with name 'CBR0' does not exist. Creating a new experiment.


bestTest = 4.957027783
bestIteration = 2999
Training on fold [0/5]


[W 2022-10-14 06:41:21,614] Trial 1 failed because of the following error: CatBoostError("C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/cuda/methods/boosting_progress_tracker.cpp:171: Saved model's params are different from current model's params")
Traceback (most recent call last):
  File "c:\Users\nehcr\miniconda3\envs\ml\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\nehcr\AppData\Local\Temp\ipykernel_23356\3352166716.py", line 24, in __call__
    cv_result = cv(pool=pool,
  File "c:\Users\nehcr\miniconda3\envs\ml\lib\site-packages\catboost\core.py", line 6650, in cv
    return _cv(params, pool, fold_count, inverted, partition_random_seed, shuffle, stratified,
  File "_catboost.pyx", line 5494, in _catboost._cv
  File "_catboost.pyx", line 5526, in _catboost._cv
_catboost.CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/cuda/methods/boosti

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/cuda/methods/boosting_progress_tracker.cpp:171: Saved model's params are different from current model's params

In [ ]:
CBR_study.best_params

{'loss_function': 'RMSE',
 'learning_rate': 0.29668232997002686,
 'l2_leaf_reg': 32.44296096601971,
 'use_best_model': False,
 'depth': 16}

### using validation set

#### make pool

In [4]:
# train test split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

# make Pool
train_pool = Pool(X_train, y_train, cat_features=cat_features, timestamp=X_train['시간'])
valid_pool = Pool(X_valid, y_valid, cat_features=cat_features, timestamp=X_valid['시간'])
test_pool = Pool(test, cat_features=cat_features, timestamp=test['시간'])

#### search

In [8]:
class Objective(object):
    def __init__(self, pool):
        self.pool = pool

    def __call__(self, trial):
        pool = self.pool
        
        CBR_params = {# prefix
                     'task_type' : 'GPU',
                     'random_seed' : seed,
                     'iterations' : 3000,
                     'early_stopping_rounds' : 50,

                     # found
                     # 'loss_function' : trial.suggest_categorical("loss_function", ['MAPE', 'Poisson', 'RMSE']),
                     'loss_function' : 'RMSE',

                     # trial
                     'learning_rate' : trial.suggest_uniform("learning_rate", 1e-4, 3e-1),
                     'depth' : trial.suggest_int("depth", 2, 16),
                     'l2_leaf_reg' : trial.suggest_float("l2_leaf_reg", 1e-5, 5e-1, log=True),
                      }
        
        # Generate model
        model = CatBoostRegressor(**CBR_params)
        model.fit(train_pool, eval_set=valid_pool, verbose=False, metric_period=100)

        # 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
        rmse = model.get_best_score()["validation"]["RMSE"]
        return rmse

CBR_objective = Objective(train_pool)

# random sampler
sampler = TPESampler(seed=seed)

CBR_study = optuna.create_study(direction="minimize", pruner=optuna.pruners.HyperbandPruner(max_resource="auto"), sampler=sampler)

[I 2022-10-14 12:21:58,067] A new study created in memory with name: no-name-f6bcc208-c8c5-46c9-8df2-ef0a724d6ad2


In [10]:
def make_mlflow_callback():
    cb = MLflowCallback(
        tracking_uri="mlruns",
        metric_name="RMSLE"
    )
    return cb

mlflow_cb = make_mlflow_callback()
CBR_study.optimize(CBR_objective, n_trials=30, callbacks=[mlflow_cb])

# data 폴더로 가서 mlflow ui 실행
# http://localhost:5000/ 로 접속

[I 2022-10-14 12:35:14,033] Trial 1 finished with value: 5.026081570380106 and parameters: {'learning_rate': 0.23690023944415856, 'depth': 3, 'l2_leaf_reg': 1.3023994368569839e-05}. Best is trial 1 with value: 5.026081570380106.
2022/10/14 12:35:14 INFO mlflow.tracking.fluent: Experiment with name 'no-name-f6bcc208-c8c5-46c9-8df2-ef0a724d6ad2' does not exist. Creating a new experiment.
[I 2022-10-14 13:19:08,758] Trial 2 finished with value: 4.432348261732767 and parameters: {'learning_rate': 0.10759363753797235, 'depth': 9, 'l2_leaf_reg': 0.0063877799620465445}. Best is trial 2 with value: 4.432348261732767.
[I 2022-10-14 14:35:40,964] Trial 3 finished with value: 4.286220289636413 and parameters: {'learning_rate': 0.08636848108440431, 'depth': 13, 'l2_leaf_reg': 0.029230613940806335}. Best is trial 3 with value: 4.286220289636413.
[I 2022-10-14 16:18:35,322] Trial 4 finished with value: 4.28192096199695 and parameters: {'learning_rate': 0.05041997866257283, 'depth': 16, 'l2_leaf_reg'

KeyboardInterrupt: 

In [11]:
CBR_study.best_params

{'learning_rate': 0.05041997866257283,
 'depth': 16,
 'l2_leaf_reg': 3.354771769479418e-05}

## visualization

In [12]:
# 시각화
optuna.visualization.plot_optimization_history(CBR_study)

In [13]:
# 파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(CBR_study)

In [14]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(CBR_study)

## fit & predict CV

In [15]:
# remake train
X_train = train.drop(['target'], axis=1)
y_train = train['target']

cv = KFold(n_splits=5, shuffle=True, random_state=seed)

scores = []
models = []

is_holdout = False
for tri, vai in cv.split(X_train):
    print("="*50)
    #Create an instance with tuned hyperparameters
    model = CatBoostRegressor(# prefix
                              task_type = 'GPU',
                              random_seed = seed,
                              iterations = 5000,
                              early_stopping_rounds = 100,

                              # found
                              loss_function = 'RMSE',
                              
                              # trial
                              learning_rate = CBR_study.best_params['learning_rate'],
                              l2_leaf_reg = CBR_study.best_params['l2_leaf_reg'],
                              depth = CBR_study.best_params['depth']
                              )
    # make pool
    train_pool_for_cv = Pool(X_train.iloc[tri], y_train.iloc[tri], cat_features=cat_features, timestamp=X_train.iloc[tri]['시간'])
    valid_pool_for_cv = Pool(X_train.iloc[vai], y_train.iloc[vai], cat_features=cat_features, timestamp=X_train.iloc[vai]['시간'])

    model.fit(train_pool_for_cv, eval_set=valid_pool_for_cv, verbose=100, metric_period=100)
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["RMSE"])
    
    if is_holdout:
        break    

CatBoostError: bad allocation

In [17]:
scores

[4.286733894665547, 4.257945545899376, 4.261384909421786]

## CV result

In [ ]:
print(scores)
print(np.mean(scores))

In [ ]:
pred_list = []
scores = []
for i,(tri, vai) in enumerate( cv.split(x_train) ):
    pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
    pred = np.where(pred >= threshold , 1, 0)
    score = f1_score(y_train[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(x_test)[:, 1]
    pred_list.append(pred)
print(scores)
print(np.mean(scores))

In [ ]:
pred = np.mean( pred_list , axis = 0 )
pred = np.where(pred >= threshold , 1, 0)

In [ ]:
# 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['target'] = pred
sample_submission.to_csv("./submit.csv", index = False)

sample_submission